# Momentum Indicators
Confirming the crossover before trading helps avoid entering too early.

Instead of buying just because RSI is under 30 or selling just because RSI is above 70, we wait for RSI to cross back through those levels.

## Commented by:
Bill Cochrane — thank you so much for your insight! 🙏

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from ta.momentum import RSIIndicator
import ccxt
import time

In [ ]:
# Bincance API Connecting
binance = ccxt.binance({
    'apiKey': 'apiKey',
    'secret': 'secret',
    'enableRateLimit': True
})

In [ ]:
def fetch_ohlcv_full(symbol, timeframe='5m', since=None, limit=1500, max_retries=5):
    all_candles = []
    since = since or binance.parse8601('2022-08-01T00:00:00Z')  # default: 1 year ago
    while True:
        try:
            candles = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
            if not candles:
                break
            all_candles.extend(candles)

            # print(f"Fetched {len(candles)} candles since {binance.iso8601(since)}")

            # Move to next batch
            since = candles[-1][0] + 1  # Avoid duplicate timestamps

            # Respect rate limits
            time.sleep(binance.rateLimit / 1000)

        except ccxt.NetworkError as e:
            print("Network error:", e)
            max_retries -= 1
            if max_retries <= 0:
                break
            time.sleep(2)
        except Exception as e:
            print("Error:", e)
            break

    df = pd.DataFrame(all_candles, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df.set_index('Timestamp', inplace=True)
    return df


In [ ]:
# Fetch 1 year of 5-minute BTC/USDT candles
df = fetch_ohlcv_full('BTC/USDT', timeframe='4H')
print(df.head())
print(f"Total candles fetched: {len(df)}")

In [ ]:
initial_balance = 10000  # Starting capital
risk_per_trade_pct = 1   # Risk 1% per trade
stop_loss_pct = 0.01     # 1% stop-loss
take_profit_pct = 0.02   # 2% take-profit

In [ ]:
# Calculating RSI 
rsi = RSIIndicator(close=df["Close"], window=14)
df["RSI"] = rsi.rsi()
# preview the result
print(df[["Close", "RSI"]].tail(10))

# Logic Update

## Entry Conditions: 
Long only when RSI crosses up through 30: rsi_prev < 30 and rsi >= 30.
Short only when RSI crosses down through 70: rsi_prev > 70 and rsi <= 70.

## Exit Conditions: 
if the RSI reaches the 50 mark for both 

In [ ]:
# Trade tracking
in_position = False
position_type = None  # 'long' or 'short'
trades = []

for i in range(1, len(df)):

    if pd.isna(df["RSI"].iloc[i-1]) or pd.isna(df["RSI"].iloc[i]):
        continue

    rsi_prev = df["RSI"].iloc[i-1]
    rsi = df["RSI"].iloc[i]
    price = df["Close"].iloc[i]
    time = df.index[i]

    # Entry condition as Long & crossover 
    if not in_position and (rsi_prev < 30 and rsi >= 30):
        entry_price = price
        entry_time = time
        stop_loss = entry_price * (1 - stop_loss_pct)
        take_profit = entry_price * (1 + take_profit_pct)

        dollar_risk = initial_balance * (risk_per_trade_pct / 100)
        risk_per_unit = entry_price - stop_loss
        position_size = dollar_risk / risk_per_unit

        in_position = True
        position_type = "long"

    elif not in_position and (rsi_prev > 70 and rsi <= 70):
        entry_price = price
        entry_time = time
        stop_loss = entry_price * (1 + stop_loss_pct)
        take_profit = entry_price * (1 - take_profit_pct)

        dollar_risk = initial_balance * (risk_per_trade_pct / 100)
        risk_per_unit = stop_loss - entry_price
        position_size = dollar_risk / risk_per_unit

        in_position = True
        position_type = "short"


    # --- Exit Conditions ---
    elif in_position:
        exit_reason = None

        if position_type == "long":
            if price <= stop_loss:
                exit_reason = "Stop Loss"
            elif price >= take_profit:
                exit_reason = "Take Profit"
            elif rsi > 50:  # exit when momentum fades
                exit_reason = "RSI Exit"

        elif position_type == "short":
            if price >= stop_loss:
                exit_reason = "Stop Loss"
            elif price <= take_profit:
                exit_reason = "Take Profit"
            elif rsi < 50:  # exit when momentum fades
                exit_reason = "RSI Exit"

        if exit_reason:
            exit_price = price
            exit_time = time
            pnl_pct = (exit_price - entry_price) / entry_price if position_type == "long" else (entry_price - exit_price) / entry_price
            pnl_dollars = pnl_pct * entry_price * position_size

            trades.append({
                "Entry Time": entry_time,
                "Exit Time": exit_time,
                "Entry Price": entry_price,
                "Exit Price": exit_price,
                "PnL %": pnl_pct * 100,
                "PnL ($)": pnl_dollars,
                "Position Size": position_size,
                "Position Type": position_type,
                "Exit Reason": exit_reason
            })

            in_position = False
            position_type = None

In [ ]:
trades_df = pd.DataFrame(trades)
print(trades_df.tail())
print(f"Total Trades: {len(trades_df)}")

# Total PnL in dollars
total_profit = trades_df["PnL ($)"].sum()
print(f"Total Profit: ${total_profit:.2f}")

# Final capital = starting capital + total profit
final_capital = initial_balance + total_profit
print(f"Final Capital: ${final_capital:.2f}")

# Adding win rate 
win_rate = (trades_df["PnL ($)"] > 0).mean() * 100
print(f"Win Rate: {win_rate:.2f}%")

# Average Win & Loss
average_win = trades_df[trades_df["PnL ($)"] > 0]["PnL ($)"].mean()
average_loss = trades_df[trades_df["PnL ($)"] < 0]["PnL ($)"].mean()
print(f"Average Win: ${average_win:.2f}")
print(f"Average Loss: ${average_loss:.2f}")

# Profit Factor
total_gain = trades_df[trades_df["PnL ($)"] > 0]["PnL ($)"].sum()
total_loss = abs(trades_df[trades_df["PnL ($)"] < 0]["PnL ($)"].sum())
profit_factor = total_gain / total_loss if total_loss != 0 else float('inf')
print(f"Profit Factor: {profit_factor:.2f}")

# Equity curve
trades_df["Equity"] = initial_balance + trades_df["PnL ($)"].cumsum()
trades_df["Peak"] = trades_df["Equity"].cummax()
trades_df["Drawdown"] = (trades_df["Equity"] - trades_df["Peak"]) / trades_df["Peak"]
max_drawdown = trades_df["Drawdown"].min()
print(f"Max Drawdown: {max_drawdown:.2%}")

# Calculate daily returns using trade equity
trades_df["Return"] = trades_df["Equity"].pct_change()
mean_return = trades_df["Return"].mean()
std_return = trades_df["Return"].std()
sharpe_ratio = (mean_return / std_return) * (252**0.5) if std_return != 0 else 0
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(trades_df["Exit Time"], trades_df["Equity"], color='blue')
plt.title("Equity Curve – Backtested RSI Strategy (1-Year, 5m BTC/USDT)")
plt.xlabel("Time")
plt.ylabel("Capital ($)")
plt.grid(True)
plt.tight_layout()
plt.savefig("rsi_equity_curve.png")
